<a href="https://colab.research.google.com/github/Jagriti1891/JAGRITI-BALLA/blob/main/Restaurant_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime, timedelta

# ==========================================
# 1. CONFIGURATION & SETUP
# ==========================================
# We use a preset style for beautiful plots
plt.style.use('ggplot')

print("--- INITIALIZING RESTAURANT MANAGEMENT SYSTEM ---")
print("Generating realistic simulation data using NumPy...")

# ==========================================
# 2. DATA GENERATION (The "Database")
# ==========================================

# Menu Items and Prices
menu_data = {
    'Category': ['Main', 'Main', 'Main', 'Main', 'Starter', 'Starter', 'Drink', 'Drink', 'Dessert'],
    'Item': ['Spicy Ramen', 'Truffle Burger', 'Margherita Pizza', 'Steak Frites',
             'Garlic Bread', 'Caesar Salad', 'Craft Cola', 'Lemonade', 'Cheesecake'],
    'Price': [12.50, 15.00, 11.00, 22.00, 5.50, 7.00, 3.00, 3.50, 6.50]
}

# Number of orders to simulate
NUM_ORDERS = 500

# Generate random dates for the last 30 days
end_date = datetime.now()
start_date = end_date - timedelta(days=30)
date_range = pd.date_range(start=start_date, end=end_date, freq='H')

# --- Simulation using NumPy ---
# Randomly select items from the menu index
menu_indices = np.arange(len(menu_data['Item']))
random_selections = np.random.choice(menu_indices, size=NUM_ORDERS)

# Randomly select quantities (1 to 4 items per order) using NumPy
random_quantities = np.random.randint(1, 5, size=NUM_ORDERS)

# Randomly select timestamps from our date range
random_dates = np.random.choice(date_range, size=NUM_ORDERS)
random_dates = np.sort(random_dates) # Sort by date for realism

# ==========================================
# 3. DATA PROCESSING (Using Pandas)
# ==========================================
print("Processing data with Pandas...")

# Create the main DataFrame
df = pd.DataFrame({
    'Order_ID': range(1, NUM_ORDERS + 1),
    'Date': random_dates,
    'Item_Index': random_selections,
    'Quantity': random_quantities
})

# Map menu details into the main dataframe
df['Item'] = df['Item_Index'].map(lambda x: menu_data['Item'][x])
df['Category'] = df['Item_Index'].map(lambda x: menu_data['Category'][x])
df['Unit_Price'] = df['Item_Index'].map(lambda x: menu_data['Price'][x])

# Calculate Total Bill for each order
df['Total_Bill'] = df['Quantity'] * df['Unit_Price']

# Extract just the date (without time) for daily grouping
df['Day'] = df['Date'].dt.date

# ==========================================
# 4. ANALYTICS REPORT
# ==========================================

total_revenue = df['Total_Bill'].sum()
avg_order_value = df['Total_Bill'].mean()
top_item = df.groupby('Item')['Quantity'].sum().idxmax()
busiest_day = df.groupby('Day')['Total_Bill'].sum().idxmax()

print("\n" + "="*40)
print("       RESTAURANT MONTHLY REPORT       ")
print("="*40)
print(f"Total Revenue:      ${total_revenue:,.2f}")
print(f"Total Orders:       {NUM_ORDERS}")
print(f"Avg Order Value:    ${avg_order_value:.2f}")
print(f"Most Popular Item:  {top_item}")
print(f"Busiest Date:       {busiest_day}")
print("="*40 + "\n")

# ==========================================
# 5. VISUALIZATION (Using Matplotlib)
# ==========================================
print("Generating visualizations...")

# Create a figure with 2 subplots (1 row, 2 columns)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle(f'Restaurant Analytics Dashboard (Last 30 Days)', fontsize=16)

# --- CHART 1: Revenue by Category (Bar Chart) ---
category_sales = df.groupby('Category')['Total_Bill'].sum().sort_values(ascending=False)
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99']

category_sales.plot(kind='bar', ax=ax1, color=colors[:len(category_sales)], edgecolor='black')
ax1.set_title('Revenue by Category')
ax1.set_ylabel('Revenue ($)')
ax1.set_xlabel('Category')
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# Add value labels on top of bars
for i, v in enumerate(category_sales):
    ax1.text(i, v + 10, f"${v:,.0f}", ha='center', fontsize=9)

# --- CHART 2: Daily Revenue Trend (Line Chart) ---
daily_sales = df.groupby('Day')['Total_Bill'].sum()

ax2.plot(daily_sales.index, daily_sales.values, marker='o', linestyle='-', color='#2c3e50', linewidth=2)
ax2.set_title('Daily Revenue Trend')
ax2.set_ylabel('Total Sales ($)')
ax2.set_xlabel('Date')
ax2.tick_params(axis='x', rotation=45)
ax2.fill_between(daily_sales.index, daily_sales.values, color='#2c3e50', alpha=0.1)

plt.tight_layout()
print("Dashboards created successfully!")
plt.show()